# importar datos

In [1]:
import numpy as np
import matplotlib.pyplot as plt
#===========================================================================================
dat2f0=np.loadtxt("ESPECTRO1_2F.txt",float,usecols=(0,1))
dat2f1=np.loadtxt("ESPECTRO1_2F_2.txt",float,usecols=(0,1))
#==============================================================================================
datf0=np.loadtxt("ESPECTRO1_F_2.txt",float,usecols=(0,1))
datf1=np.loadtxt("ESPECTRO1_F_3.txt",float,usecols=(0,1))

# crear areglos

In [2]:
londa=dat2f0[10:,0]
sen2f0=dat2f0[10:,1]
sen2f1=dat2f1[10:,1]
#=================================
londa2=datf0[:,0]
senf0=datf0[:,1]
senf1=datf1[:,1]

# promedio

In [21]:
prom2f=(sen2f0+sen2f1)/2
promf=(senf0+senf1)/2

# Aplicando campo 

In [23]:
#===========================================================================================
dat2f0M=np.loadtxt("ESPECTRO1_2F_200mT.txt",float,usecols=(0,1))
dat2f1M=np.loadtxt("ESPECTRO1_2F_200mT.txt",float,usecols=(0,1))
#==============================================================================================
datf0M=np.loadtxt("ESPECTRO1_F_2_200mT.txt",float,usecols=(0,1))
datf1M=np.loadtxt("ESPECTRO1_F_2_200mT.txt",float,usecols=(0,1))

sen2f0M=dat2f0M[10:,1]
sen2f1M=dat2f1M[10:,1]
#=================================
senf0M=datf0M[:,1]
senf1M=datf1M[:,1]

prom2fM=(sen2f0M+sen2f1M)/2
promfM=(senf0M+senf1M)/2

In [25]:
dif2f=prom2fM-prom2f

In [31]:
datosK=np.stack((londa, promf, promfM, prom2f, prom2fM), axis=-1)

# datos BS

In [32]:
# abrir archivo con datos del psi y delta
np.savetxt("datKerr.txt",datosK)

In [33]:
datBS=np.loadtxt("DatosBS45.txt",float,usecols=(0,1,2))

In [38]:
Apsi=datBS[8:,1]
Adelt=datBS[8:,2]

In [49]:
b1=np.stack((promf,prom2f),axis=-1)
b2=np.stack((promfM,prom2fM),axis=-1)

In [40]:
from scipy import special

#parametros de la medicion

fase=2.405

j0=special.jv(0,fase)
j1=special.jv(1,fase)
j2=special.jv(2,fase)
#solucion de sistema de ecuaciones usnado numpy
def sol(psi,delta, i1):
    psirad=(psi*np.pi)/180; deltarad=(delta*np.pi)/180
    a11=4*j1*np.tan(psirad)*np.sin(deltarad); a12= -4*j1*np.tan(psirad)*np.cos(deltarad)
    a21=-4*j2*np.tan(psirad)*np.cos(deltarad); a22=-4*j2*np.tan(psirad)*np.sin(deltarad)
    a=np.array([[a11, a12],[a21 ,a22]])
    sol=np.linalg.solve(a,i1)
    angulo=sol[0]; elip=sol[1]
    return angulo, elip
#creacion de arreglos con la informacion
def arr(b):
    ang=np.zeros(Apsi.size); elip=np.zeros(Apsi.size)
    for i in range(Apsi.size):
        ang[i], elip[i] = sol(Apsi[i],Adelt[i],b[i,:])
    return ang, elip

In [41]:
thetab1, elipb1= arr(b1)

In [43]:
thetab2, elipb2= arr(b2)

In [45]:
diftheta=thetab2-thetab1
difelip=elipb2-elipb1

In [48]:
#guardado de la informacion
datosKcor=np.stack((londa, thetab1, elipb1, thetab2, elipb2), axis=-1)
np.savetxt("datKerrCorr.txt",datosKcor)